# Answers to Udacity Quiz Questions on the Enron Dataset

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# show all outputs of a cell (such as if df.head() and df.tail() are in the same cell)
#default is 'last_expr'

In [5]:
import pandas as pd
import pickle
import scipy
from pandasql import *
from pandasql import sqldf

In [3]:
os.getcwd() #use this location in loading the pickled file provided by Udacity

'/Users/Work/Desktop/Work/Projects/Machine Learning Engineer Nanodegree'

In [14]:
import pickle
enron = pickle.load(open("/Users/Work/Desktop/Work/Projects/Machine Learning Engineer Nanodegree\
/ud120-projects/final_project/final_project_dataset.pkl", "r"))

In [20]:
#how many people are in the dataset?
len(enron.keys()) 

146

In [32]:
#how many features per person?  Let's look at the number of features for the first couple people.
enron.keys()[:2] 
len(enron['METTS MARK']), len(enron['BAXTER JOHN C'])

['METTS MARK', 'BAXTER JOHN C']

(21, 21)

In [111]:
#Count the number of persons of interest (feature: POI)
df=pd.DataFrame(enron).T.reset_index()
df.head()
df.poi.sum()

,index,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,ALLEN PHILLIP K,4175000,2869717,-3081055,NaN,phillip.allen@enron.com,1729541,13868,2195,47,...,304805,152,False,126027,-126027,201955,1407,2902,4484442,1729541
1,BADUM JAMES P,NaN,178980,NaN,NaN,NaN,257817,3486,NaN,NaN,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,182466,257817
2,BANNANTINE JAMES M,NaN,NaN,-5104,NaN,james.bannantine@enron.com,4046157,56301,29,39,...,NaN,864523,False,1757552,-560222,477,465,566,916197,5243487
3,BAXTER JOHN C,1200000,1295738,-1386055,NaN,NaN,6680544,11200,NaN,NaN,...,1586055,2660303,False,3942714,NaN,267102,NaN,NaN,5634343,10623258
4,BAY FRANKLIN R,400000,260455,-201641,NaN,frank.bay@enron.com,NaN,129142,NaN,NaN,...,NaN,69,False,145796,-82782,239671,NaN,NaN,827696,63014


18

In [70]:
#How many POI's were there in total (not just those contained in the dataset, but who Udacity identified as POI's?)
file = open('/Users/Work/Desktop/Work/Projects/Machine Learning Engineer Nanodegree/\
ud120-projects/final_project/poi_names.txt','rt')
txt = file.read()
file.close()
txt=txt.splitlines()
txt[:4]     
txt=txt[2:]    #remove first two entries, which aren't POI
txt2=[]
for j in txt:
    txt2.append(j[4:])
txt2[:3] #notice that the (y)/(n) markers are removed (this was not necessary to determine that there are 35 POI's)
len(txt2)

['http://usatoday30.usatoday.com/money/industries/energy/2005-12-28-enron-participants_x.htm',
 '',
 '(y) Lay, Kenneth',
 '(y) Skilling, Jeffrey']

['Lay, Kenneth', 'Skilling, Jeffrey', 'Howard, Kevin']

35

In [112]:
#What is the total value of the stock belonging to James Prentice?
df.rename(columns={'index':'person'}, inplace=True)
df[df.person.str.contains('PRENTICE')].total_stock_value

109    1095040
Name: total_stock_value, dtype: object

In [114]:
#How many email messages do we have from Wesley Colwell to persons of interest?
df.columns #figure out which column provides us info for this question...
df[df.person.str.contains('WESLEY')].from_this_person_to_poi 

Index([u'person', u'bonus', u'deferral_payments', u'deferred_income',
       u'director_fees', u'email_address', u'exercised_stock_options',
       u'expenses', u'from_messages', u'from_poi_to_this_person',
       u'from_this_person_to_poi', u'loan_advances', u'long_term_incentive',
       u'other', u'poi', u'restricted_stock', u'restricted_stock_deferred',
       u'salary', u'shared_receipt_with_poi', u'to_messages',
       u'total_payments', u'total_stock_value'],
      dtype='object')

26    11
Name: from_this_person_to_poi, dtype: object

In [116]:
#What’s the value of stock options exercised by Jeffrey K Skilling?
df[df.person.str.contains('SKILLING')].exercised_stock_options

122    19250000
Name: exercised_stock_options, dtype: object

In [144]:
#Of these three individuals (Lay, Skilling and Fastow), who took home the most money 
#(largest value of “total_payments” feature)?  How much money did that person get?
df[df.person.str.contains('SKILLING|LAY|FASTOW')][['person','total_payments']].sort_values('total_payments', \
                                                                                           ascending=False)[:1]

,person,total_payments
79,LAY KENNETH L,103559793


In [150]:
#How are features denoted when they are missing a value?  answer: with NaN
df[:1]

,person,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,ALLEN PHILLIP K,4175000,2869717,-3081055,NaN,phillip.allen@enron.com,1729541,13868,2195,47,...,304805,152,False,126027,-126027,201955,1407,2902,4484442,1729541


In [165]:
#How many folks in this dataset have a quantified salary? What about a known email address?
len(df[df.salary!='NaN'].salary)
len(df[df.email_address!='NaN'].salary)

95

111

In [176]:
#How many people in the E+F dataset (as it currently exists) have “NaN” for their total payments? 
#What percentage of people in the dataset as a whole is this?
len(df[df.total_payments=='NaN'].total_payments)/float(len(df))

0.14383561643835616

In [186]:
#If a machine learning algorithm were to use total_payments as a feature,
#would you expect it to associate a “NaN” value with POIs or non-POIs? 
#answer: NaN is only associated with non_POI's.  So a NaN would be predictive of the observation being a non-POI.
sql = lambda q: sqldf(q, globals()) 

q='''
SELECT POI, COUNT(*) 
FROM df
WHERE total_payments='NaN'
GROUP BY POI
'''
sql(q)

,poi,COUNT(*)
0,0,21
